importamos las librerias necesarias.

In [71]:
#importamos paquetes
import numpy as np
import pandas as pd
import datetime as dt

analizamos el dataset

In [72]:
#cargamos el data y le damos un vistazo
check = pd.read_csv('/OnlineRetail.csv')
check.head()

,InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country
0,536365;85123A;WHITE HANGING HEART T-LIGHT HOLD...
1,536365;71053;WHITE METAL LANTERN;6;12/1/2010 8...
2,536365;84406B;CREAM CUPID HEARTS COAT HANGER;8...
3,536365;84029G;KNITTED UNION FLAG HOT WATER BOT...
4,536365;84029E;RED WOOLLY HOTTIE WHITE HEART.;6...


el data esta separado por ; se debe cambiar por comas para continuar.

In [73]:
#vovlemos a cargar el data con la nueva instruccion.
check1 = pd.read_csv('/OnlineRetail.csv', sep=';')
check1.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


eliminamos el formato de hora en la columna de las fechas para no tomarlas encuenta.

In [74]:
check1['InvoiceDate'] = check1['InvoiceDate'].apply(lambda x: x.split(' ')[0])
check1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010,3.39,17850.0,United Kingdom


ahora vemos la cantidad de filas y columnas, y si hay valores nulos en ellas.

In [75]:
#vemos las cantitadesde filas y columnas
print('Numero de filas: ', check1.shape[0])
print('Numero de columnas: ', check1.shape[1])

Numero de filas:  541909
Numero de columnas:  8


In [76]:
#vemos cantidades y localizaciones de numeros nulos
check1.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

ahora que sabemos que hay valores nulos, los eliminamos para que no interfieran en el procedimiento para cumplir con los requisitos del proyecto.

In [77]:
#eliminamos valores nulos
check1.dropna(inplace=True)


In [78]:
#comprobamos la eliminacion de valores nulos
check1.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

ahora veremos si existe algun valor que resulte ilogico mediante la descripcion del data.

In [79]:
#vemos los resultados de desviacion estandar
check1.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


en la columna de cantidades resulta en valor negativo, y es ilogico en en el mundo exista una cantidad bajo 0 de algo, dado que esa cantidad por si misma indica que no hay nada; eliminamos los valores negativos para que quede limpio el data.

In [80]:
#eliminamos los valores negativos de la columna Quantity y comprobamos
check2 = check1[check1.Quantity >= 0]
check2.describe()

,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


ahora se prodcedera a calcular los valores RFM que solicita el proyecto

In [84]:
# calculamos la frecuencia de los clientes
import datetime as dt
check2.loc[:,['InvoiceDate']] = pd.to_datetime(check2['InvoiceDate'])
check_recency = check2.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max()
check_recency.columns = ['CustomerID','LastPurchaseDate']
last_date = check_recency['LastPurchaseDate'].max()
check_recency['Recency'] = check_recency['LastPurchaseDate'].apply(lambda x: (last_date - x).days)
check_recency.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,CustomerID,LastPurchaseDate,Recency
0,12346.0,2011-01-18,325
1,12347.0,2011-12-07,2
2,12348.0,2011-09-25,75
3,12349.0,2011-11-21,18
4,12350.0,2011-02-02,310


In [86]:
#calculamos el valor de la frecuencia
frequency_df = check2.drop_duplicates().groupby(
    by=['CustomerID'], as_index=False)['InvoiceDate'].count() #Agrupamos por CustomerID tomando datos de fechas de visita sin considerar duplicados.
frequency_df.columns = ['CustomerID', 'Frequency'] #Nombramos columnas.
frequency_df.head()

,CustomerID,Frequency
0,12346.0,1
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


In [87]:
#calculamos por ultimo el valor monetario
check2['Total'] = check2['UnitPrice']*check2['Quantity'] # Calculamos totales multiplicando precio unitario * customerID
monetary_df = check2.groupby(by='CustomerID', as_index=False)['Total'].sum() #Agrupamos por CustomerID
monetary_df.columns = ['CustomerID', 'Monetary'] #Creamos columnas del nuevo dataframe.
monetary_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


una vez obtenidos los valores se incluiran en una tabla para una mejor percepcion de los datos.

In [91]:
ultimatecheck = pd.DataFrame([monetary_df.CustomerID,
                    check_recency.Recency,
                    frequency_df.Frequency,
                    monetary_df.Monetary]).transpose()

In [92]:
ultimatecheck.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325.0,1.0,77183.60
1,12347.0,2.0,182.0,4310.00
2,12348.0,75.0,31.0,1797.24
3,12349.0,18.0,73.0,1757.55
4,12350.0,310.0,17.0,334.40


Tenemos estos valores y  el ultimo paso nos pide que les demos un un score, con el que podamos segmentar los perfiles, para este caso usaremos una escala del 1 al 5. 

In [99]:
quantiles = ultimatecheck.quantile(q=[0.2,0.4,0.6,0.8]) #Definimos las proporciones de nuestros cuantiles.
quantiles = quantiles.to_dict() #Los almacenamos en un diccionario. 

In [100]:
segmented_rfm = ultimatecheck #Creamos un segundo dataframe a partir del primer dataframe.

In [101]:
def RScore(x,p,d): #Creamos una función para asignarle un valor, score, a los valores de Recientes, mientras menor sea el valor, es decir más reciente, mejor
    if x <= d[p][0.2]:
        return 1 #Si el dato es igual o menor que el 20% entra en valor 1
    elif x <= d[p][0.4]:
        return 2 #Si el dato es igual o menor que el 40% entra en valor 2
    elif x <= d[p][0.6]: 
        return 3 ##Si el dato es igual o menor que el 60% entra en valor 3
    elif x <= d[p][0.8]:
        return 4 #Si el dato es igual o menor que el 80% entra en valor 4
    else:
        return 5 #Y para todo lo demás, lo mayor al 80%, entra en valor 5
    


In [102]:
def FMScore(x,p,d): #Repetimos proceso pero para Frecuencia y valor monetario, en este caso mientras mayor sea la frecuencia, mejor. 
    if x <= d[p][0.2]:
        return 5
    elif x <= d[p][0.4]:
        return 4
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]:
        return 2
    else:
        return 1

In [105]:
ultimatecheck['r_quartile'] = ultimatecheck['Recency'].apply(RScore, args=('Recency',quantiles,)) #Aplicamos la función Rscore
segmented_rfm['f_quartile'] = segmented_rfm['Frequency'].apply(FMScore, args=('Frequency',quantiles,)) #Aplicamos la función FMscore
segmented_rfm['m_quartile'] = segmented_rfm['Monetary'].apply(FMScore, args=('Monetary',quantiles,)) #Aplicamos la función FMscore.
segmented_rfm.head(15) #Obtenemos una gráfica donde ya tenemos los scores o puntuaciones de cada columna.

,CustomerID,Recency,Frequency,Monetary,r_quartile,f_quartile,m_quartile
0,12346.0,325.0,1.0,77183.60,5,5,1
1,12347.0,2.0,182.0,4310.00,1,1,1
2,12348.0,75.0,31.0,1797.24,4,3,2
3,12349.0,18.0,73.0,1757.55,2,2,2
4,12350.0,310.0,17.0,334.40,5,4,4
5,12352.0,36.0,85.0,2506.04,3,2,1
6,12353.0,204.0,4.0,89.00,5,5,5
7,12354.0,232.0,58.0,1079.40,5,2,2
8,12355.0,214.0,13.0,459.40,5,5,4
9,12356.0,22.0,59.0,2811.43,2,2,1
